# Nested JSON data transformation

In [299]:
import pandas as pd
import json

In [300]:
pd.read_json('data.json')

,students
0,"{'student_id': 1, 'name': 'John Smith', 'cours..."
1,"{'student_id': 2, 'name': 'Alice Johnson', 'co..."
2,"{'student_id': 3, 'name': 'Bob Davis', 'course..."


In [301]:
with open('data.json') as file:
    data = json.load(file)

In [302]:
data

{'students': [{'student_id': 1,
   'name': 'John Smith',
   'courses': [{'course_id': 101, 'course_name': 'Math', 'grade': 95},
    {'course_id': 102, 'course_name': 'Science', 'grade': 88}]},
  {'student_id': 2,
   'name': 'Alice Johnson',
   'courses': [{'course_id': 101, 'course_name': 'Math', 'grade': 92},
    {'course_id': 103, 'course_name': 'History', 'grade': 78}]},
  {'student_id': 3,
   'name': 'Bob Davis',
   'courses': [{'course_id': 102, 'course_name': 'Science', 'grade': 87},
    {'course_id': 103, 'course_name': 'History', 'grade': 85}]}]}

In [303]:
data['students']

[{'student_id': 1,
  'name': 'John Smith',
  'courses': [{'course_id': 101, 'course_name': 'Math', 'grade': 95},
   {'course_id': 102, 'course_name': 'Science', 'grade': 88}]},
 {'student_id': 2,
  'name': 'Alice Johnson',
  'courses': [{'course_id': 101, 'course_name': 'Math', 'grade': 92},
   {'course_id': 103, 'course_name': 'History', 'grade': 78}]},
 {'student_id': 3,
  'name': 'Bob Davis',
  'courses': [{'course_id': 102, 'course_name': 'Science', 'grade': 87},
   {'course_id': 103, 'course_name': 'History', 'grade': 85}]}]

In [304]:
df = pd.DataFrame(data['students'])
df

,student_id,name,courses
0,1,John Smith,"[{'course_id': 101, 'course_name': 'Math', 'gr..."
1,2,Alice Johnson,"[{'course_id': 101, 'course_name': 'Math', 'gr..."
2,3,Bob Davis,"[{'course_id': 102, 'course_name': 'Science', ..."


In [305]:
df = df.explode('courses')
df

,student_id,name,courses
0,1,John Smith,"{'course_id': 101, 'course_name': 'Math', 'gra..."
0,1,John Smith,"{'course_id': 102, 'course_name': 'Science', '..."
1,2,Alice Johnson,"{'course_id': 101, 'course_name': 'Math', 'gra..."
1,2,Alice Johnson,"{'course_id': 103, 'course_name': 'History', '..."
2,3,Bob Davis,"{'course_id': 102, 'course_name': 'Science', '..."
2,3,Bob Davis,"{'course_id': 103, 'course_name': 'History', '..."


In [306]:
df = df.explode('courses').reset_index(drop=True)
df

,student_id,name,courses
0,1,John Smith,course_id
1,1,John Smith,course_name
2,1,John Smith,grade
3,1,John Smith,course_id
4,1,John Smith,course_name
5,1,John Smith,grade
6,2,Alice Johnson,course_id
7,2,Alice Johnson,course_name
8,2,Alice Johnson,grade
9,2,Alice Johnson,course_id


In [307]:
pd.json_normalize(data['students'])

,student_id,name,courses
0,1,John Smith,"[{'course_id': 101, 'course_name': 'Math', 'gr..."
1,2,Alice Johnson,"[{'course_id': 101, 'course_name': 'Math', 'gr..."
2,3,Bob Davis,"[{'course_id': 102, 'course_name': 'Science', ..."


In [308]:
df = pd.DataFrame(data['students'])  # 1
df = df.explode('courses')  # 2
pd.json_normalize(df['courses'])  # 3

,course_id,course_name,grade
0,101,Math,95
1,102,Science,88
2,101,Math,92
3,103,History,78
4,102,Science,87
5,103,History,85


In [309]:
pd.merge(df, pd.json_normalize(df['courses']), left_index=True, right_index=True)  # 4

,student_id,name,courses,course_id,course_name,grade
0,1,John Smith,"{'course_id': 101, 'course_name': 'Math', 'gra...",101,Math,95
0,1,John Smith,"{'course_id': 102, 'course_name': 'Science', '...",101,Math,95
1,2,Alice Johnson,"{'course_id': 101, 'course_name': 'Math', 'gra...",102,Science,88
1,2,Alice Johnson,"{'course_id': 103, 'course_name': 'History', '...",102,Science,88
2,3,Bob Davis,"{'course_id': 102, 'course_name': 'Science', '...",101,Math,92
2,3,Bob Davis,"{'course_id': 103, 'course_name': 'History', '...",101,Math,92


In [310]:
pd.merge(df, pd.json_normalize(df['courses']), left_index=True, right_index=True).drop(columns=['courses'])  # 5

,student_id,name,course_id,course_name,grade
0,1,John Smith,101,Math,95
0,1,John Smith,101,Math,95
1,2,Alice Johnson,102,Science,88
1,2,Alice Johnson,102,Science,88
2,3,Bob Davis,101,Math,92
2,3,Bob Davis,101,Math,92
